In [1]:
import parser 
import pandas as pd
import itertools
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from voting_rules import voting_rules as vr
import rcv_dimensionality
import statistics
import rcv_distribution

In [2]:
def is_consistent(ballot):
    if len(ballot) == 0: 
        return (True, None) 
    if len(ballot) == 1:
        return (True, ballot[0]) 
    x = 0 
    v = 0.25
    i = 1
    while (i < len(ballot)):
        if ballot[i] < ballot[i - 1]:
            x -= (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        else:
            x += (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        v *= 0.5
        i += 1
    l = []
    if abs(x) >= (ballot[1] - ballot[0])/2 or x == 0:
        return (False, x + ballot[0]) 
    x += ballot[0]
    for c in ballot:
        l.append(abs(c - x))
    if(all(l[i] <= l[i + 1] for i in range(len(l) -  1))):
        return (True, x)
    return (False, x)

In [3]:
filename = "dataverse_files/Alaska_08162022_HouseofRepresentativesSpecial.csv"
ballots, candidates = rcv_distribution.parse_election_data(filename)
mds = rcv_dimensionality.perform_rcv_and_normalize(filename)


/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:171: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


['Begich, Nick', 'Palin, Sarah', 'Peltola, Mary S.']


In [14]:
axis = mds.keys()
order = mds.keys()
temp = {}
i = 1
for c in order:
    temp[c] = i
    i += 1

position = {}
x = 1/(len(axis) + 1)
v = x = 1/(len(axis) + 1)
for c in axis:
    position[c] = x
    x += v
print(position)
X = position.values()
#non bullet votes 
bullet = {}
nb = {}
for b in ballots:
    ballot = []
    for c in b:
        ballot.append(temp[c])
    if len(b) > 1 and is_consistent(ballot)[0] is True:
        first = b[0]
        x = is_consistent(ballot)[1]
        if x < position[first]:
            if position[first] - v/4 not in nb:
                nb[position[first] - v/4] = 0
            nb[position[first] - v/4] += ballots[b]
        else:
            if position[first] + v/4 not in nb:
                nb[position[first] +v/4] = 0
            nb[position[first] + v/4] += ballots[b]
        
    if len(b) == 1:
        first = b[0]
        if first not in nb:
            bullet[first] = 0
        bullet[first] += ballots[b]


{'Palin, Sarah': 0.25, 'Begich, Nick': 0.5, 'Peltola, Mary S.': 0.75}


In [15]:
print(bullet)
print(nb)

{'Peltola, Mary S.': 23660, 'Palin, Sarah': 21186, 'Begich, Nick': 11208}
{0.5625: 15507, 0.3125: 34899}
